In [4]:
import numpy as np
import os

In [14]:
CurrentPath = os.getcwd()
data_1 = np.loadtxt(CurrentPath + '/KASI_Ms/2025/data_1.txt', delimiter=' ')
data_2 = np.loadtxt(CurrentPath + '/KASI_Ms/2025/data_2.txt', delimiter=' ')

### Fitting with function

In [71]:
def func_1(x,parameter): # define a quadratic function
    return parameter[0]*x**2 + parameter[1]*x + parameter[2]
def chisq(func, x, y, var,params = None): # returns chi-square value for the estimation
    dof = len(x)
    if params is not None:
        arr_exp = func(x,params)
    else:
        arr_exp = func(x)
    chisquared = np.sum((y - arr_exp)**2/var)
    return chisquared

In [73]:
from scipy.optimize import minimize

data_1_X = data_1.T[0];data_1_Y = data_1.T[1]; data_1_var = data_1.T[2]
data_2_X = data_2.T[0];data_2_Y = data_2.T[1]; data_2_var = data_2.T[2]
def objective1(params): # creates a function that can be put into scipy.optimize.minimize as an argument
    return chisq(func_1, data_1_X, data_1_Y, data_1_var, params)
def objective2(params):
    return chisq(func_1, data_2_X, data_2_Y, data_2_var, params)


In [74]:
initial = np.array([4,3,1])
result1= minimize(objective1, initial)
result2 = minimize(objective2, initial)

###

In [75]:
result1

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 11763694233.78489
        x: [ 1.028e+02 -3.809e+03  2.820e+04]
      nit: 8
      jac: [ 4.992e+03  2.560e+02  0.000e+00]
 hess_inv: [[ 6.291e-09 -4.838e-07  2.966e-06]
            [-4.838e-07  3.994e-05 -2.445e-04]
            [ 2.966e-06 -2.445e-04  1.496e-03]]
     nfev: 241
     njev: 58

In [84]:
print(f"""
      data_1 result of chisq fitting:
      success : {result1.success} \\
      chisq per dof : {result1.fun / len(data_1_X)}
      p-value :
      """)
print(fr"func : ${result1.x[0]:.2f}x^2$ + ${result1.x[1]:.2f}x$ + ${result1.x[2]:.2f}$")
print(f"""
      data_1 result of chisq fitting:
      success : {result2.success}
      chisq per dof : {result2.fun / len(data_2_X)}
      p-value :
      """)
print(fr"func : ${result2.x[0]:.2f}x^2$ + ${result2.x[1]:.2f}x$ + ${result2.x[2]:.2f}$")


      data_1 result of chisq fitting:
      success : False \
      chisq per dof : 470547769.35139555
      p-value :
      
func : $102.84x^2$ + $-3808.76x$ + $28199.59$

      data_1 result of chisq fitting:
      success : True
      chisq per dof : 0.2103654906193602
      p-value :
      
func : $11.54x^2$ + $1.99x$ + $4.67$


### Plotting two results

0.21036549062054472

In [57]:
len(data_1_X)

25